In [1]:
from openfisca_us import IndividualSim, Microsimulation
import pandas as pd
import numpy as np

In [2]:
sim = IndividualSim(year=2022)
sim.add_person(name="parent", age=30, employment_income=2_000*12)
sim.add_person(name="child", age=3)
MEMBERS = ["parent", "child"]
sim.add_tax_unit(name="tax_unit", members=MEMBERS)
sim.add_spm_unit(name="spm_unit", members=MEMBERS, snap_emergency_allotment=0)
sim.add_household(name="household", members=MEMBERS, state_code="MA")

In [3]:
sim.calc("snap")

array([349.1997], dtype=float32)

In [4]:
sim.calc("cdcc")

array([0.], dtype=float32)

In [5]:
def make_df(childcare_expenses):
    sim = IndividualSim(year=2022)
    sim.add_person(name="parent", age=30, would_claim_wic=True, is_wic_at_nutritional_risk=True)
    sim.add_person(name="child", age=3, would_claim_wic=True, is_wic_at_nutritional_risk=True)
    MEMBERS = ["parent", "child"]
    sim.add_tax_unit(name="tax_unit", members=MEMBERS, premium_tax_credit=0)
    sim.add_spm_unit(name="spm_unit", members=MEMBERS, snap_emergency_allotment=0,
                     childcare_expenses=childcare_expenses)
    sim.add_household(name="household", members=MEMBERS, state_code="MA")
    sim.vary("employment_income", max=500_000)

    return pd.DataFrame(dict(
        employment_income=sim.calc("employment_income")[0],
        net_income=sim.calc("spm_unit_net_income")[0],
        childcare_expenses=childcare_expenses,
        snap=sim.calc("snap")[0],
        cdcc=sim.calc("cdcc")[0]
    ))

In [6]:
l = []
for i in [0, 500 * 12, 1_000 * 12]:
    l.append(make_df(i))
df = pd.concat(l)

In [7]:
import plotly.express as px

px.line(df, "employment_income", "net_income", color="childcare_expenses")

In [8]:
px.line(df, "employment_income", "cdcc", color="childcare_expenses")

In [9]:
wide = df.pivot(index="employment_income", columns="childcare_expenses")
wide["net_income_diff"] = wide.net_income[12000] - wide.net_income[0]

In [10]:
px.line(x=wide.index, y=wide.net_income_diff)

In [11]:
36/120

0.3

## Microsimulation

Create a reform that repeals CDCC and SNAP childcare deduction.

In [12]:
from openfisca_us.model_api import *
from openfisca_us.tools.baseline_variables import baseline_variables

class repeal_cdcc_snap_childcare(Reform):
    def apply(self):
        self.neutralize_variable("cdcc")
        self.neutralize_variable("snap_dependent_care_deduction")

In [13]:
msb = Microsimulation(year=2020)
msr = Microsimulation(repeal_cdcc_snap_childcare, year=2020)

In [14]:
msr.calc("spm_unit_net_income").sum() - msb.calc("spm_unit_net_income").sum()

0.0